In [0]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [0]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [5]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

word_vectors=KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary='True')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
v_apple=word_vectors["apple"]
v_orange=word_vectors["orange"]

In [0]:
print(v_apple.shape,v_orange.shape)

(300,) (300,)


In [0]:
cosine_similarity([v_apple],[v_orange])

array([[0.33449447]], dtype=float32)

## 1 ODD ONE OUT

In [0]:
# accepts list of words and return odd one out
def odd_one_out(words):
  # Generate all word embeddings for the given list
  all_word_vectors=[word_vectors[word] for word in words]
  avg_vec=np.mean(all_word_vectors,axis=0)
  print(avg_vec.shape)
  #iterate over every words and find similarity
  odd_one=None
  min_similarity=1
  for w in words:
    sim=cosine_similarity([word_vectors[w]],[avg_vec])
    if sim<min_similarity:
      min_similarity=sim
      odd_one=w
    print("Similarity between %s and average vector is %.2f"%(w,sim))
  return odd_one    


In [0]:
input_1 = ["apple","mango","juice","party","orange"] 
input_2 = ["music","dance","sleep","dancer","food"]        
input_3  = ["match","player","football","cricket","dancer"]
input_4 = ["india","paris","russia","france","germany"]

In [0]:
odd_one_out(input_2)

(300,)
Similarity between music and average vector is 0.66
Similarity between dance and average vector is 0.81
Similarity between sleep and average vector is 0.51
Similarity between dancer and average vector is 0.72
Similarity between food and average vector is 0.52


'sleep'

### 2. Word Analogies Task

In [0]:
type(word_vectors.vocab)

dict

In [0]:
#accepts a triad a,b,c then predict d such that when a is to b::c is to d
def predict_word(a,b,c,word_vectors):
  a,b,c=a.lower(),b.lower(),c.lower()
  max_similarity=-100
  d=None
  words=word_vectors.vocab.keys()
  wa,wb,wc=word_vectors[a],word_vectors[b],word_vectors[c]
  for w in words:
    if w in [a,b,c]:
      continue
    wv=word_vectors[w]
    sim=cosine_similarity([wb-wa],[wv-wc])
    if sim>max_similarity:
      max_similarity=sim
      d=w
  return d

In [8]:
triad=["man","woman","king"]
print(predict_word(*triad,word_vectors))

queen


In [6]:
# We can also find word analogies using most similar method
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071)]